Neste Notebook o dataset dos vistos disponível na internet, pelo link abaixo, e que contem mais de 3 milhões de linhas é modificado para um novo dataset. 

https://www.kaggle.com/nsharan/h-1b-visa

In [1]:
import pandas as pd
import numpy as np
import random
from pandas import ExcelWriter

Na célula abaixo, o dataset é importado e após isso as linhas são embaralhadas. 

In [2]:
dataset = pd.read_csv("h1b.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2902141,CERTIFIED,"SMART COMPUTERS AND CONSULTING, LLC",Computer Systems Analysts,COMPUTER ANALYST,Y,43846.0,2011.0,"DICKINSON, NORTH DAKOTA",-102.789624,46.879176
1,1537113,CERTIFIED,"TRINAMIX, INC.",Computer Systems Analysts,SYSTEMS ANALYST,Y,62088.0,2014.0,"SANTA ANA, CALIFORNIA",-117.867834,33.745573
2,1188476,CERTIFIED,AMERICAN BANKERS INSURANCE COMPANY OF FLORIDA,COMPUTER SYSTEMS ANALYSTS,BUSINESS ANALYST III,Y,78000.0,2015.0,"MIAMI, FLORIDA",-80.191790,25.761680
3,1819039,CERTIFIED,WIPRO LIMITED,Computer Programmers,PROGRAMMER ANALYST,Y,63898.0,2013.0,"ALPHARETTA, GEORGIA",-84.294090,34.075376
4,968732,CERTIFIED,UHCP D/B/A UPMC MEP,"HEALTH DIAGNOSING AND TREATING PRACTITIONERS, ALL",FELLOW - MOVEMENT DISORDERS,Y,59407.0,2015.0,"PITTSBURGH, PENNSYLVANIA",-79.995886,40.440625


Na célula seguinte a um tratamento de dados. Todas as linhas na qual o resultado é WITHDRAWN, são retiradas do dataset, isso pois ele não cai em numa categoria que queira ser estudada, e também pois o processo não foi terminado. As duas linas de código seguintas também retiram linhas do dataset, pelo mesmo motivo a cima, o processo de aplicação para o visto de trabalho não foi terminado. Logo após isso as variavés são transformadas em 0 ou 1, para quando aprovado ou reprovado (CERTIFIED OU DENIED). Vale ressaltar que o caso CERTIFIED-WITHDRAWN, significa que a pessoa foi aprovada mas não quis terminar o processo para obter o visto, esse caso foi considerado como aprovado para o nosso estudo. Por último todas linhas qua não haviam uma das colunas completas, são removidas.

In [3]:
dataset = dataset[dataset.CASE_STATUS != "WITHDRAWN"]
dataset = dataset[dataset.CASE_STATUS != "PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED"]
dataset = dataset[dataset.CASE_STATUS != "INVALIDATED"]

dataset["CASE_STATUS"] = dataset["CASE_STATUS"].apply(lambda x: 1 if (x=="CERTIFIED-WITHDRAWN" or x=="CERTIFIED") else 0)

dataset = dataset.dropna()
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2902141,1,"SMART COMPUTERS AND CONSULTING, LLC",Computer Systems Analysts,COMPUTER ANALYST,Y,43846.0,2011.0,"DICKINSON, NORTH DAKOTA",-102.789624,46.879176
1,1537113,1,"TRINAMIX, INC.",Computer Systems Analysts,SYSTEMS ANALYST,Y,62088.0,2014.0,"SANTA ANA, CALIFORNIA",-117.867834,33.745573
2,1188476,1,AMERICAN BANKERS INSURANCE COMPANY OF FLORIDA,COMPUTER SYSTEMS ANALYSTS,BUSINESS ANALYST III,Y,78000.0,2015.0,"MIAMI, FLORIDA",-80.191790,25.761680
3,1819039,1,WIPRO LIMITED,Computer Programmers,PROGRAMMER ANALYST,Y,63898.0,2013.0,"ALPHARETTA, GEORGIA",-84.294090,34.075376
4,968732,1,UHCP D/B/A UPMC MEP,"HEALTH DIAGNOSING AND TREATING PRACTITIONERS, ALL",FELLOW - MOVEMENT DISORDERS,Y,59407.0,2015.0,"PITTSBURGH, PENNSYLVANIA",-79.995886,40.440625


Abaixo o novos datasets são criados a partir da aprovação ou reprovação no visto e depois são cortados na linha 50000. Por fim esse dois novos dataset são concatenados em um outro, o qual é exportado para um arquivo csv

In [4]:
na = dataset[dataset["CASE_STATUS"] == 0]
ap = dataset[dataset["CASE_STATUS"] == 1]

na_cut = na[:50000]
ap_cut = ap[:50000]

df = pd.concat([ap_cut, na_cut])
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,824764,1,ERNST & YOUNG U.S. LLP,COMPUTER SYSTEMS ANALYSTS,ADVISORY MANAGER,Y,103667.0,2015.0,"HOUSTON, TEXAS",-95.369803,29.760427
1,50636,1,CORNING OPTICAL COMMUNICATIONS LLC,MARKET RESEARCH ANALYSTS AND MARKETING SPECIAL...,STRATEGIC PLANNING ANALYST,Y,82139.0,2016.0,"HICKORY, NORTH CAROLINA",-81.344457,35.734454
2,1258118,1,SOCIAL MEDIA EDUCATION GROUP PTY LTD,OPERATIONS RESEARCH ANALYSTS,VICE PRESIDENT OF CLIENT SERVICES,Y,64210.0,2015.0,"SAN FRANCISCO, CALIFORNIA",-122.419415,37.774929
3,2691274,0,GENPACT PROCESS SOLUTIONS LLC,Accountants,ASSISTANT MANAGER,Y,62546.0,2011.0,"STAMFORD, CONNECTICUT",-73.538734,41.053430
4,2680263,0,"NETWORK OBJECTS, INC.",Computer Programmers,PROGRAMMER ANALYST,Y,63586.0,2011.0,"HOFFMAN ESTATES, ILLINOIS",-88.122720,42.062991


In [5]:
df.to_csv("Dataframe.csv")